In [2]:
import os

def find_glpsol():
    # Common directories to search
    possible_directories = [
        os.path.expanduser("~/AppData/Local/Anaconda3/Library/bin/"),  # Conda default location
        os.path.expanduser("~/AppData/Local/Anaconda3/envs/"),         # Conda environments
        "C:/Program Files/GLPK/bin/",                    # Typical Program Files location
        "C:/Program Files (x86)/GLPK/bin/",              # Another potential Program Files location
    ]
    
    # Search through Conda environments
    conda_env_path = os.path.expanduser("~/AppData/Local/Anaconda3/envs/")
    if os.path.exists(conda_env_path):
        for env in os.listdir(conda_env_path):
            possible_directories.append(os.path.join(conda_env_path, env, "Library/bin/"))
    
    # Check if glpsol.exe exists in any of the possible directories
    for directory in possible_directories:
        glpsol_path = os.path.join(directory, "glpsol.exe").replace("\\","/")
        if os.path.isfile(glpsol_path):
            return glpsol_path

    # Search system PATH as a last resort
    for path in os.environ["PATH"].split(os.pathsep):
        glpsol_path = os.path.join(path, "glpsol.exe")
        if os.path.isfile(glpsol_path):
            return glpsol_path

    return None

# Run the script
glpsol_path = find_glpsol()

if glpsol_path:
    print(f"Found glpsol.exe at: {glpsol_path}")
else:
    print("glpsol.exe not found on the system.")


Found glpsol.exe at: C:/Users/hali/AppData/Local/Anaconda3/envs/myenv/Library/bin/glpsol.exe


In [42]:
#Original Model w/ Sensitivity Analysis Report
import pulp
from pulp import GLPK_CMD, LpProblem, LpMaximize, LpVariable, LpStatus

# Define raw materials and products
raw_materials = ['RSXX', 'STYY', 'CBAA', 'CBZZ', 'SBRXL', 'SLSM', 'CARBON 10', 'CARBON 20', 'CARBON 30', 'CARBON 40', 'CARBON 60']
products = ['Product 1', 'Product 2', 'Product 8B*', 'Product 3', 'Product 4', 'Product 4*', 'Product 10', 'Product 6', 'Product 9']

# Define data as arrays
raw_material_costs = [0.40, 0.60, 1.1, 1.1, 2.0, 2.0, 8.5, 5.0, 6.0, 3.5, 6.0]
product_prices = [90, 70, 100, 75, 80, 100, 150, 90, 120]
site1_inventory = [3000, 4000, 1000, 500, 200, 900, 1500, 2000, 800, 800, 500]
site2_inventory = [5000, 2500, 1750, 250, 400, 1200, 1250, 1900, 800, 800, 250]

# Define raw material usage per product (rows: raw materials, columns: products)
raw_material_usage = [
    [0.50, 0.30, 0.00, 0.33, 0.25, 0.53, 0.53, 0.25, 0.66],  # RSXX
    [0.00, 0.00, 0.50, 0.00, 0.25, 0.00, 0.00, 0.25, 0.00],  # STYY
    [0.15, 0.12, 0.13, 0.13, 0.00, 0.00, 0.00, 0.15, 0.00],  # CBAA
    [0.00, 0.00, 0.00, 0.00, 0.13, 0.13, 0.13, 0.00, 0.00],  # CBZZ
    [0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # SBRXL
    [0.00, 0.00, 0.00, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],  # SLSM
    [0.35, 0.00, 0.37, 0.00, 0.37, 0.34, 0.00, 0.00, 0.00],  # CARBON 10
    [0.00, 0.00, 0.00, 0.34, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 20
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35, 0.00],  # CARBON 30
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.34, 0.00, 0.34],  # CARBON 40
    [0.00, 0.38, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],  # CARBON 60
]

# Define labor hours per product at each site
site1_labor_hours = [0.25, 0.25, 0.35, 0.25, 0.25, 0.35, 0.25, 0.25, 0.20]
site2_labor_hours = [0.35, 0.35, 0.50, 0.35, 0.35, 0.50, 0.35, 0.35, 0.35]

# Define the problem
prob = pulp.LpProblem("Maximize_Profit", pulp.LpMaximize)

# Define decision variables for the quantity of each product to produce at each site
product_vars_site1 = [pulp.LpVariable(f"Product{i+1}_Site1", lowBound=0, cat='Continuous') for i in range(len(products))]
product_vars_site2 = [pulp.LpVariable(f"Product{i+1}_Site2", lowBound=0, cat='Continuous') for i in range(len(products))]

# Calculate profit per product per site
profit_values_site1 = []
profit_values_site2 = []

for i in range(len(products)):
    cost = 15 * sum(raw_material_costs[j] * raw_material_usage[j][i] for j in range(len(raw_materials)))
    profit_values_site1.append(product_prices[i] - cost)
    profit_values_site2.append(product_prices[i] - cost)

profits_site1 = [product_vars_site1[i] * profit_values_site1[i] for i in range(len(products))]
profits_site2 = [product_vars_site2[i] * profit_values_site2[i] for i in range(len(products))]

# Objective function
prob += pulp.lpSum(profits_site1 + profits_site2), "Total_Profit"

# Calculate total labor hours per product per site
labor_hours_site1 = [product_vars_site1[i] * site1_labor_hours[i] for i in range(len(products))]
labor_hours_site2 = [product_vars_site2[i] * site2_labor_hours[i] for i in range(len(products))]

# Sum the total labor hours for each site
sum_labor_site1 = pulp.lpSum(labor_hours_site1)
sum_labor_site2 = pulp.lpSum(labor_hours_site2)

# Add labor hours constraints
prob += sum_labor_site1 <= 168, "Site1_Labor_Constraint"
prob += sum_labor_site2 <= 168, "Site2_Labor_Constraint"

# Calculate inventory constraints per raw material per site
for i in range(len(raw_materials)):
    # Constraint for Site 1
    prob += pulp.lpSum(product_vars_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site1_inventory[i], f"Site1_{raw_materials[i]}_Constraint"
    
    # Constraint for Site 2
    prob += pulp.lpSum(product_vars_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) <= site2_inventory[i], f"Site2_{raw_materials[i]}_Constraint"

# Solve the problem using GLPK
glpk_path = "C:/Users/hali/AppData/Local/Anaconda3/envs/myenv/Library/bin/glpsol.exe"
status = prob.solve(GLPK_CMD(path=glpk_path, options=['--ranges', 'Product_Optimization.sen']))

# Output results
optimal_values_site1 = [product_vars_site1[i].varValue for i in range(len(products))]
optimal_values_site2 = [product_vars_site2[i].varValue for i in range(len(products))]
objective_value = pulp.value(prob.objective)

# Calculate inventory used per raw material
inventory_used_site1 = {raw_materials[i]: sum(optimal_values_site1[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) for i in range(len(raw_materials))}
inventory_used_site2 = {raw_materials[i]: sum(optimal_values_site2[j] * raw_material_usage[i][j] * 15 for j in range(len(products))) for i in range(len(raw_materials))}

# Calculate total labor used
total_labor_used_site1 = sum([optimal_values_site1[i] * site1_labor_hours[i] for i in range(len(products))])
total_labor_used_site2 = sum([optimal_values_site2[i] * site2_labor_hours[i] for i in range(len(products))])

# Print results
print("Optimal production values for Site 1:", optimal_values_site1)
print("Optimal production values for Site 2:", optimal_values_site2)
print("Total Profit:", objective_value)

print("Inventory used per raw material at Site 1:")
for material, usage in inventory_used_site1.items():
    print(f"{material}: {usage}")

print("Inventory used per raw material at Site 2:")
for material, usage in inventory_used_site2.items():
    print(f"{material}: {usage}")

print("Total labor used at Site 1:", total_labor_used_site1)
print("Total labor used at Site 2:", total_labor_used_site2)

# To extract sensitivity analysis, read the generated .sen file
try:
    with open("Product_Optimization.sen", "r") as f:
        sensitivity_analysis = f.read()
        print("\nSensitivity Analysis:\n")
        print(sensitivity_analysis)
except FileNotFoundError:
    print("Sensitivity analysis file not found. Ensure GLPK has generated the sensitivity report.")

Optimal production values for Site 1: [0.0, 0.0, 88.6192, 238.689, 0.0, 0.0, 156.863, 152.381, 0.0]
Optimal production values for Site 2: [170.756, 0.0, 0.0, 0.0, 0.0, 0.0, 128.205, 152.381, 28.6576]
Total Profit: 75203.050825
Inventory used per raw material at Site 1:
RSXX: 3000.00015
STYY: 1236.07275
CBAA: 981.10824
CBZZ: 305.88284999999996
SBRXL: 0.0
SLSM: 716.067
CARBON 10: 491.83656
CARBON 20: 1217.3139
CARBON 30: 800.0002499999999
CARBON 40: 800.0013
CARBON 60: 0.0
Inventory used per raw material at Site 2:
RSXX: 3155.0387400000004
STYY: 571.42875
CBAA: 727.05825
CBZZ: 249.99975
SBRXL: 0.0
SLSM: 0.0
CARBON 10: 896.4689999999999
CARBON 20: 0.0
CARBON 30: 800.0002499999999
CARBON 40: 799.9992600000002
CARBON 60: 0.0
Total labor used at Site 1: 167.99997
Total labor used at Site 2: 167.99985999999998

Sensitivity Analysis:

GLPK 5.0  - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  Total_Profit =